In [1]:
import pandas as pd
import numpy as np
from luwiji.knn import illustration, demo

In [2]:
df = pd.read_csv("data/titanic.csv", index_col= "PassengerId")
df.head()

,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
PassengerId,,,,,,,,,,,
1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [3]:
df.drop(columns=["Name","Cabin", "Ticket"], inplace = True)

In [4]:
df.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
PassengerId,,,,,,,,
1,0,3,male,22.0,1,0,7.2500,S
2,1,1,female,38.0,1,0,71.2833,C
3,1,3,female,26.0,0,0,7.9250,S
4,1,1,female,35.0,1,0,53.1000,S
5,0,3,male,35.0,0,0,8.0500,S


In [5]:
from sklearn.model_selection import train_test_split
X = df.drop(columns=["Survived"])
y = df.Survived

X_train, X_test, y_train, y_test = train_test_split(X, y, stratify = y, test_size=0.2, random_state=42)

In [6]:
X_train.head()

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
PassengerId,,,,,,,
693,3,male,NaN,0,0,56.4958,S
482,2,male,NaN,0,0,0.0000,S
528,1,male,NaN,0,0,221.7792,S
856,3,female,18.0,0,1,9.3500,S
802,2,female,31.0,1,1,26.2500,S


In [7]:
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder, StandardScaler, RobustScaler

numerical_pipeline = Pipeline([
    ("impute", SimpleImputer(strategy='mean')),
    ("scaler", RobustScaler())
])
categorical_pipeline = Pipeline([
    ("impute", SimpleImputer(strategy='most_frequent')),
    ("scaler", OneHotEncoder())
])

In [8]:
from sklearn.compose import ColumnTransformer
preprocessor = ColumnTransformer([
    ("numerik", numerical_pipeline, ["Age", "SibSp", "Parch", "Fare"]),
    ("kategorik", categorical_pipeline, ["Pclass","Sex", "Embarked"])
])

In [9]:
from sklearn.neighbors import KNeighborsClassifier

pipeline = Pipeline([
    ("preprocessor", preprocessor),
    ("algo", KNeighborsClassifier())
])

pipeline.fit(X_train, y_train)

Pipeline(memory=None,
         steps=[('preprocessor',
                 ColumnTransformer(n_jobs=None, remainder='drop',
                                   sparse_threshold=0.3,
                                   transformer_weights=None,
                                   transformers=[('numerik',
                                                  Pipeline(memory=None,
                                                           steps=[('impute',
                                                                   SimpleImputer(add_indicator=False,
                                                                                 copy=True,
                                                                                 fill_value=None,
                                                                                 missing_values=nan,
                                                                                 strategy='mean',
                                                            

In [10]:
pipeline.score(X_train, y_train), pipeline.score(X_test, y_test)

(0.8567415730337079, 0.770949720670391)

## GridSearchCV

In [11]:
from sklearn.model_selection import GridSearchCV

In [12]:
pipeline.get_params()

{'memory': None,
 'steps': [('preprocessor',
   ColumnTransformer(n_jobs=None, remainder='drop', sparse_threshold=0.3,
                     transformer_weights=None,
                     transformers=[('numerik',
                                    Pipeline(memory=None,
                                             steps=[('impute',
                                                     SimpleImputer(add_indicator=False,
                                                                   copy=True,
                                                                   fill_value=None,
                                                                   missing_values=nan,
                                                                   strategy='mean',
                                                                   verbose=0)),
                                                    ('scaler',
                                                     RobustScaler(copy=True,
                         

In [13]:
parameter = {
    "algo__n_neighbors" : range(1, 51, 2),
    "algo__weights": ["uniform", "distance"],
    "algo__p" : [1, 2, 3]
}

model = GridSearchCV(pipeline, parameter, n_jobs = -1, cv = 3, verbose = 1)
model.fit(X_train, y_train)

Fitting 3 folds for each of 150 candidates, totalling 450 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:    8.8s
[Parallel(n_jobs=-1)]: Done 184 tasks      | elapsed:   11.5s
[Parallel(n_jobs=-1)]: Done 450 out of 450 | elapsed:   16.1s finished


GridSearchCV(cv=3, error_score=nan,
             estimator=Pipeline(memory=None,
                                steps=[('preprocessor',
                                        ColumnTransformer(n_jobs=None,
                                                          remainder='drop',
                                                          sparse_threshold=0.3,
                                                          transformer_weights=None,
                                                          transformers=[('numerik',
                                                                         Pipeline(memory=None,
                                                                                  steps=[('impute',
                                                                                          SimpleImputer(add_indicator=False,
                                                                                                        copy=True,
                                 

In [14]:
model.best_params_

{'algo__n_neighbors': 9, 'algo__p': 2, 'algo__weights': 'uniform'}

In [15]:
model.score(X_train, y_train), model.score(X_test, y_test)

(0.8328651685393258, 0.7877094972067039)

In [19]:
X_train.head()

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
PassengerId,,,,,,,
693,3,male,NaN,0,0,56.4958,S
482,2,male,NaN,0,0,0.0000,S
528,1,male,NaN,0,0,221.7792,S
856,3,female,18.0,0,1,9.3500,S
802,2,female,31.0,1,1,26.2500,S


In [20]:
data = [
    [1, 'female', 20, 1, 1, 80, "S"],
    [1, 'male', 21,0,0,5,"S"]
]

In [21]:
x_pred = pd.DataFrame(data, index = ['Rose', 'Jack'], columns= X.columns)
x_pred
model.predict(x_pred)

array([1, 0], dtype=int64)